# 1. ResNet

## $\bullet$ packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms
import torch
import torch.nn as nn
import os
from tqdm import tqdm
import gc
%matplotlib inline


In [ ]:
cuda = torch.cuda.is_available()

## $\bullet$ path

In [ ]:
train_image_dir = '../input/shopee-product-matching/train_images'
file_list = os.listdir(train_image_dir)

## $\bullet$ transform images to fixed size 224

In [ ]:
#transform image to fixed size 
target_size = 224
image_transform = transforms.Resize(target_size)

def plt_image(image_name):
    image = Image.open(train_image_dir + '/' + image_name)
    image = image_transform(image)
    #image = np.array(image)
    #print("image_shape: ", image.shape)
    #plt.imshow(image)
    #plt.show()
    image = transforms.ToTensor()(image)
    return image

## $\bullet$ RetNet50

In [ ]:
class base_resnet(nn.Module):
    def __init__(self):
        super(base_resnet, self).__init__()
        self.model = models.resnet50(pretrained=True)
    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)
        x = self.model.avgpool(x)

        return x

In [ ]:
resnet_model = base_resnet()
if cuda:
    resnet_model.cuda()

## $\bullet$ extract features

In [ ]:
image_train = []
for i in tqdm(range(int(len(file_list)))):
    image = plt_image(file_list[i])
    image_train.append(image)

In [ ]:
image_train = torch.stack(image_train, dim=0)
if cuda:
    image_train.cuda()
image_feature = resnet_model(image_train)

In [ ]:
image_feature.shape